In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

C:\Users\asus\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
df = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)

In [6]:
df.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [7]:
df.shape

(748, 2)

In [8]:
imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
yelp = pd.read_csv('yelp_labelled.txt',sep='\t',header=None)

In [11]:
# import os

In [12]:
# os.listdir()

In [13]:
df = pd.DataFrame()

In [14]:
df = df.append(imdb)
df = df.append(amazon)
df = df.append(yelp)

In [16]:
df.shape

(2748, 2)

In [17]:
df.columns = ["Review","Sentiment"]

In [18]:
df.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [23]:
df['Sentiment'].iloc[0]

0

In [28]:
def text_processing(df):
    table = str.maketrans("","",string.punctuation)
    for i in range(len(df)):
        df['Review'].iloc[i] = df['Review'].iloc[i].translate(table)
    
    for i in range(len(df)):
        df['Review'].iloc[i] = word_tokenize(df['Review'].iloc[i].lower())
        
    eng_stopwords = stopwords.words("english")
    words = []
    for i in range(len(df)):
        token_list = []
        for token in df['Review'].iloc[i]:
            if token not in eng_stopwords:
                token_list.append(token)
        words.append(token_list)
        
    wnet = WordNetLemmatizer()
    for i in range(len(words)):
        for j in range(len(words[i])):
            words[i][j] = wnet.lemmatize(words[i][j],pos='v')
            
    for i in range(len(words)):
        words[i] = " ".join(words[i])
    
    return words

In [29]:
wordsList = text_processing(df)

C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
wordsList[2]

'attempt artiness black white clever camera angle movie disappoint become even ridiculous act poor plot line almost nonexistent'

In [33]:
cv = CountVectorizer()

In [34]:
vector = cv.fit_transform(wordsList).toarray()

In [36]:
vector.shape

(2748, 4532)

In [38]:
x_train,x_test,y_train,y_test = train_test_split(vector,df['Sentiment'])

In [40]:
x_train.shape

(2061, 4532)

In [39]:
model = Sequential()

In [42]:
# input layer + hidden layer_1
model.add(Dense(units=1024, input_dim=x_train.shape[1], init='uniform', activation='relu'))
# hidden_layer_2
model.add(Dense(units=512,activation='relu', init='uniform'))
# output_layer
model.add(Dense(units=1, activation='sigmoid', init='uniform'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,batch_size=32)

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1024, input_dim=4532, activation="relu", kernel_initializer="uniform")`
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=512, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`
  


Epoch 1/10
2061/2061 [==============================] - 7s 3ms/step - loss: 0.5743 - accuracy: 0.7098
Epoch 2/10
2061/2061 [==============================] - 6s 3ms/step - loss: 0.1597 - accuracy: 0.9427
Epoch 3/10
2061/2061 [==============================] - 7s 3ms/step - loss: 0.0447 - accuracy: 0.9859
Epoch 4/10
2061/2061 [==============================] - 7s 4ms/step - loss: 0.0194 - accuracy: 0.9932
Epoch 5/10
2061/2061 [==============================] - 8s 4ms/step - loss: 0.0122 - accuracy: 0.9961
Epoch 6/10
2061/2061 [==============================] - 7s 4ms/step - loss: 0.0103 - accuracy: 0.9961
Epoch 7/10
2061/2061 [==============================] - 8s 4ms/step - loss: 0.0087 - accuracy: 0.9971
Epoch 8/10
2061/2061 [==============================] - 8s 4ms/step - loss: 0.0084 - accuracy: 0.9971
Epoch 9/10
2061/2061 [==============================] - 9s 4ms/step - loss: 0.0075 - accuracy: 0.9971
Epoch 10/10
2061/2061 [==============================] - 8s 4ms/step - loss: 0.006

In [43]:
y_pred = model.predict(x_test)

In [44]:
from sklearn.metrics import accuracy_score

In [47]:
y_pred = y_pred > 0.5

In [49]:
accuracy_score(y_test,y_pred)

0.7903930131004366

In [50]:
model.save('ann_model.h5')

In [58]:
new_text = "To say that War has a wafer-thin plot is an understatement. War has a lot of visual appeal but the all style and no substance approach makes this 2 hours 36 minutes film a real drag. DISAPPOINTING! It isn't completely unwatchable, but it certainly tests your patience. How many times have you ordered a dish that looks terrific photographed in a menu but disappoints when it shows up on the table? War is that kind of meal and a golden opportunity lost. It has two handsome leads, some eye-popping stunt scenes (Paul Jennings, Franz Spilhaus, Oh Sea Young & Parvez Shaikh) and a slew of beautiful locales. It would take an especially awful script & an incompetent director to deliver a mess out of those ingredients."

In [59]:
test_df = pd.DataFrame({"Review":[new_text]})

In [60]:
test_df

,Review
0,To say that War has a wafer-thin plot is an un...


In [61]:
wordsList = text_processing(test_df)
test_vect = cv.transform(wordsList).toarray()

In [62]:
test_vect

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [64]:
pred = model.predict(test_vect)

In [65]:
pred > 0.5

array([[False]])